In [2]:
import pandas as pd
import spacy
from time import time
from gensim.models import Word2Vec
import logging

In [9]:
# dados_treino = pd.read_csv(r"..\06-word2vec-interpretação-da-linguagem-humana-com-word-embedding\database-train-project-07.csv")
# dados_teste = pd.read_csv(r"..\06-word2vec-interpretação-da-linguagem-humana-com-word-embedding\database-test-project-07.csv")

dados_teste = pd.read_csv("https://raw.githubusercontent.com/goosekiing/word2vec-interpretacao-da-linguagem-humana-com-word-embedding/eb26014ad11cf54981bc048c541f4b28f8f0337b/database-test-project-07.csv")
dados_treino = pd.concat([pd.read_csv(f"https://raw.githubusercontent.com/goosekiing/word2vec-interpretacao-da-linguagem-humana-com-word-embedding/eb26014ad11cf54981bc048c541f4b28f8f0337b/database-train-project-07-part-{i+1}.csv") for i in range(3)])

display(dados_treino.sample(3), dados_teste.sample(3))

,title,text,date,category,subcategory,link
91,"Vereadores atuam contra ciclovias, e secretári...",A vereadora Edir Sales (PSD) já organizou abai...,2017-05-27,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/05...
11241,Hiroshima recorda bombardeio e pede desarmamen...,Os japoneses relembraram neste domingo (6) o p...,2017-06-08,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/08/190...
14279,"No Rio, tratamentos de câncer são interrompido...",Quase a metade dos tratamentos de câncer na ca...,2017-03-14,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/03...


,title,text,date,category,subcategory,link
11762,Barclays vai encerrar sua cobertura doméstica ...,O Barclays iniciou uma nova rodada de corte de...,2016-01-21,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/01/1...
3637,Trump ameaça aliados que votarem contra lei qu...,Para os deputados republicanos que ainda estão...,2017-03-21,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/03/186...
18914,Não há perseguição política contra Cristina Ki...,Responsável por duas das principais denúncias ...,2016-08-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/08/180...


In [4]:
nlp = spacy.load("pt_core_news_sm")
nlp

In [5]:
texto = "O Palmeiras perdeu 1 semi final da Libertadores para o Boca Juniors em um empate por 1x1!"
doc = nlp(texto)
print(doc)
print(type(doc))

O Palmeiras perdeu 1 semi final da Libertadores para o Boca Juniors em um empate por 1x1!
<class 'spacy.tokens.doc.Doc'>


In [6]:
print(doc[1])
print(doc.ents)
print(doc[0], doc[0].is_stop)

Palmeiras
(Palmeiras, Libertadores, Boca Juniors)
O True


In [7]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [8]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        eh_valido = not token.is_stop and token.is_alpha
        if eh_valido:
            tokens_validos.append(token.text)
    
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)

In [9]:
trata_textos(doc)

'Palmeiras perdeu semi Libertadores Boca Juniors empate'

In [10]:
t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]
t1 = time()
print(f"Tempo de processamento: {(t1-t0):.2f} seg.")


Tempo de processamento: 191.81 seg.


In [11]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.sample(4)

,titulo
23473,saída crise econômica legitimar governo temer ...
54510,fluminense palmeirenses passarão vestibular
25246,al qaeda pede ataques ocidente seguidores
84565,atraso pagamento direitos empregados demitirem...


In [12]:
w2v_modelo = Word2Vec(sg=0, window=2, vector_size=300, min_count=5, alpha=0.03, min_alpha=0.007)
w2v_modelo

In [13]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))


90000
84466


In [14]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [18]:
logging.basicConfig(format="%(asctime)s : %(message)s", level=logging.INFO)
w2v_modelo = Word2Vec(sg=0, window=2, vector_size=300, min_count=5, alpha=0.03, min_alpha=0.007)
w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2023-10-24 18:52:13,489 : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2023-10-24T18:52:13.488480', 'gensim': '4.3.2', 'python': '3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2023-10-24 18:52:13,491 : collecting all words and their counts
2023-10-24 18:52:13,493 : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-10-24 18:52:13,512 : PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2023-10-24 18:52:13,530 : PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2023-10-24 18:52:13,550 : PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2023-10-24 18:52:13,578 : PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2023-10-24 18:52:13,632 : PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word t

In [21]:
w2v_modelo.corpus_count

84466

In [23]:
w2v_modelo.train(lista_lista_tokens, total_examples=w2v_modelo.corpus_count, epochs=30)

2023-10-24 19:09:17,189 : Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2023-10-24T19:09:17.189094', 'gensim': '4.3.2', 'python': '3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'train'}
2023-10-24 19:09:17,981 : EPOCH 0: training on 540242 raw words (485990 effective words) took 0.8s, 629636 effective words/s
2023-10-24 19:09:18,703 : EPOCH 1: training on 540242 raw words (486083 effective words) took 0.7s, 684860 effective words/s
2023-10-24 19:09:19,424 : EPOCH 2: training on 540242 raw words (486122 effective words) took 0.7s, 685834 effective words/s
2023-10-24 19:09:20,151 : EPOCH 3: training on 540242 raw words (486184 effective words) took 0.7s, 681328 effective words/s
2023-10-24 19:09:20,857 : EPOCH 4: training on 540242 raw words (486023 effective words

(14583621, 16207260)

In [25]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5733203291893005),
 ('tesla', 0.5593856573104858),
 ('amazon', 0.5572776794433594),
 ('viajante', 0.5125274658203125),
 ('sony', 0.5109540224075317),
 ('linkedin', 0.5102859735488892),
 ('sky', 0.5081335306167603),
 ('spotify', 0.5013250708580017),
 ('blue', 0.4991951584815979),
 ('braskem', 0.4931843876838684)]

In [24]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5866417288780212),
 ('facebook', 0.5465093851089478),
 ('amazon', 0.4762831926345825),
 ('waze', 0.4555312991142273),
 ('airbnb', 0.44368472695350647),
 ('fbi', 0.4382781386375427),
 ('software', 0.4335442781448364),
 ('yahoo', 0.4327748715877533),
 ('news', 0.42736244201660156),
 ('uber', 0.426719605922699)]

In [26]:
w2v_modelo.wv.most_similar("barcelona")

[('barça', 0.6106361746788025),
 ('madrid', 0.5756472945213318),
 ('bayern', 0.5631275773048401),
 ('psg', 0.5363635420799255),
 ('united', 0.5330449938774109),
 ('munique', 0.5267422199249268),
 ('juventus', 0.5228532552719116),
 ('chelsea', 0.5081841349601746),
 ('suárez', 0.5022984743118286),
 ('figueirense', 0.5012516975402832)]

In [27]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.5849438905715942),
 ('cristiano', 0.5306991934776306),
 ('barça', 0.5115934014320374),
 ('ronaldo', 0.5081584453582764),
 ('tevez', 0.5007955431938171),
 ('cavani', 0.4860115051269531),
 ('enrique', 0.48485511541366577),
 ('psg', 0.48254117369651794),
 ('neymar', 0.48040562868118286),
 ('chuteiras', 0.4794771373271942)]

In [28]:
w2v_modelo.wv.most_similar("gm")

[('volks', 0.6660826802253723),
 ('embraer', 0.6503634452819824),
 ('chrysler', 0.6422983407974243),
 ('honda', 0.6135537624359131),
 ('tesla', 0.6121627688407898),
 ('fiat', 0.6078920960426331),
 ('braskem', 0.5994606018066406),
 ('renault', 0.5979273915290833),
 ('toyota', 0.5936806201934814),
 ('volkswagen', 0.5867471694946289)]

In [29]:
# Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg=1, window=5, vector_size=300, min_count=5, alpha=0.03, min_alpha=0.007)
w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)
w2v_modelo_sg.train(lista_lista_tokens, total_examples=w2v_modelo_sg.corpus_count, epochs=30)

2023-10-24 19:17:17,509 : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2023-10-24T19:17:17.509807', 'gensim': '4.3.2', 'python': '3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2023-10-24 19:17:17,511 : collecting all words and their counts
2023-10-24 19:17:17,513 : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-10-24 19:17:17,536 : PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2023-10-24 19:17:17,566 : PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2023-10-24 19:17:17,594 : PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2023-10-24 19:17:17,618 : PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2023-10-24 19:17:17,638 : PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word t

(14583982, 16207260)

In [30]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.4538048505783081),
 ('android', 0.40502890944480896),
 ('waze', 0.39599043130874634),
 ('app', 0.39272433519363403),
 ('patentes', 0.3813683092594147),
 ('toshiba', 0.37972643971443176),
 ('apple', 0.3773418664932251),
 ('concorda', 0.3703386187553406),
 ('buffett', 0.3644103407859802),
 ('facebook', 0.3638223111629486)]

In [31]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5726913809776306),
 ('motors', 0.5537147521972656),
 ('bmw', 0.5085549354553223),
 ('honda', 0.49513164162635803),
 ('mitsubishi', 0.4906786382198334),
 ('audi', 0.49007976055145264),
 ('airbags', 0.4816265404224396),
 ('coletivas', 0.4765009582042694),
 ('cubatão', 0.47496771812438965),
 ('chrysler', 0.4684428870677948)]

In [33]:
w2v_modelo.wv.save_word2vec_format(fname="modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format(fname="modelo_skipgram.txt", binary=False)

2023-10-24 19:28:32,092 : storing 12924x300 projection weights into modelo_cbow.txt


2023-10-24 19:28:35,018 : storing 12924x300 projection weights into modelo_skipgram.txt
